Заглушка

Тема курсовой: "Болталка", "Бот-всезнайка"

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 61.1 MB/s eta 0:00:00


In [2]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 1.8 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [3]:
from datasets import load_dataset

dataset = load_dataset("radm/tathagata")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/radm___text/radm--tathagata-6b19756d75eb994b/0.0.0/cb1e9bd71a82ad27976be3b12b407850fe2837d80c22c5e03a28949843a8ace2. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
dataset['train'][:5]

{'text': ['Всё, наблюдаемое в мире, лишено усилий и активности, ибо всё сущее в мире подобно сну или чудесно возникшим образам. Этого не понимают философы и невежды. Но те, кто видит сущее именно таким, видит его истинно. Те же, кто воспринимает его иначе, вовлечены в различения и, будучи зависимы от них, привязаны к двойственности. Восприятие мира с позиции различения подобно изучению собственного отражения в зеркале, или собственной тени, или луны, отражённой в воде, или эха, услышанного в долине. Люди, опутанные тенями собственного различения, привязываются к той или иной вещи и, оказываясь не в состоянии избавиться от двойственности, продолжают беспрестанное различение и потому никогда не достигают успокоения. Под успокоением понимается Единость, а Единостью порождается наивысшее Самадхи, обретаемое вступлением в царство Благородной Мудрости, выявляемой лишь в глубочайших недрах сознания.',
  'Поскольку невежды и простаки, не ведающие, что мир является лишь чем-то самим умом вообра

In [5]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

model_name_or_path = "radm/rugpt3medium-tathagata"
tokenizer = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3medium_based_on_gpt2")
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [6]:
def respond_to_dialog(texts):
    prefix = '\nx:'
    for i, t in enumerate(texts):
        prefix += t
        prefix += '\nx:' if i % 2 == 1 else '\ny:'
    tokens = tokenizer(prefix, return_tensors='pt')
    tokens = {k: v.to(model.device) for k, v in tokens.items()}
    end_token_id = tokenizer.encode('\n')[0]
    size = tokens['input_ids'].shape[1]
    output = model.generate(
        **tokens, 
        eos_token_id=end_token_id,
        do_sample=True, 
        max_length=size+128, 
        repetition_penalty=3.2, 
        temperature=1,
        num_beams=3,
        length_penalty=0.01,
        pad_token_id=end_token_id
    )
    decoded = tokenizer.decode(output[0])
    result = decoded[len(prefix):]
    return result.strip()

In [7]:
seed = input('Начните диалог с ботом любой фразой\n')
history = [seed]
while True:
    result = respond_to_dialog(history[-10:])
    next_sentence = input(result + '\n')
    if next_sentence == 'пока': break
    history.append(result)
    history.append(next_sentence)

Начните диалог с ботом любой фразой
Привет
Как дела?
Супер! А у тебя ? 
у меня всё хорошо. Я просто наслаждаюсь моментом.
Я тоже 
орошо, что мы не говорим по-немецки. У нас бы получилась путаница.
Да, мы говорим на русском 
а, мы говорим друг с другом на одном языке и понимаем друг друга. Но это не одно и то же. Мы говорим друг с другом через ум, а он говорит со мной
Пока
ока
пока


In [8]:
!pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11695 sha256=3b81ba897ef08a79fdfb06af2c6e58145551321a2de99457bc00d757a11f98ee
  Stored in directory: /root/.cache/pip/wheels/07/93/05/72c05349177dca2e0ba31a33ba4f7907606f7ddef303517c6a
Successfully built wikipedia


In [9]:
import wikipedia

In [10]:
wikipedia.set_lang("ru")
wikipedia.page('Россия').content[:1000]

'Росси́я, или Росси́йская Федера́ция (сокр. РФ), — государство в Восточной Европе и Северной Азии. Россия — крупнейшее государство в мире, её территория в конституционных границах с территорией Крыма, аннексия которого не получила международного признания, составляет 17 125 191 км². Население страны в тех же границах составляет 146 424 729 чел. (2023; 9-е место в мире).\nСтолица — Москва. Государственный язык на всей территории страны — русский, в ряде регионов России также установлены свои государственные и официальные языки. Денежная единица — российский рубль.\nРоссия — многонациональное государство с широким этнокультурным многообразием. Согласно результатам переписей населения России 2010 года, а также Крыма и Севастополя 2014 года в стране живут представители свыше 190 национальностей, среди которых русские составляют свыше 80 %, а русским языком владеют свыше 99,4 % россиян. Бо́льшая часть населения (около 75 %) в религиозном отношении относит себя к православию, что делает Росс